# BUSINESS PROBLEM & UNDERSTANDING
STAKEHOLDER:SPOTIFY
PROBLEM: Spotify is launch a new Movie Divison. We are contracted Data Scientist tasked with  creating a reccomendation system . We want to compete with other streaming platforms such as Netflix, Hulu, Amazon Prime. Since they are new to the market, they want to ensure that their Reccomendation System is accurate and reliable.   

 Your task is to:

Build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies.
The MovieLens dataset is a "classic" recommendation system dataset, that is used in numerous academic papers and machine learning proofs-of-concept. You will need to create the specific details about how the user will provide their ratings of other movies, in addition to formulating a more specific business problem within the general context of "recommending movies".




# Data Preparation

In [1]:
#Importing Libraries 
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
from datetime import datetime

from collections import Counter

from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
%matplotlib inline

In [2]:
# Reading in datasets
df_movies = pd.read_csv('data/movies.csv')
df_links = pd.read_csv('data/links.csv')
df_ratings = pd.read_csv('data/ratings.csv')
df_tags = pd.read_csv('data/tags.csv')

In [3]:
#looking at each indivdual csv before merging.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [4]:
df_links.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [5]:
df_ratings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [6]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


Converting Ratings and Tags Timestamps to Datetime format

In [7]:
#df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'])
#df_tags['timestamp'] = pd.to_datetime(df_tags['timestamp'])

In [8]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [9]:
df_movies[df_movies['title'].duplicated() == True]

,movieId,title,genres
5601,26958,Emma (1996),Romance
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


Merging Dataset to one Pandas Database

In [10]:
# combining the Movies, Ratings, Tags Dataset. Not including Links dataset due to its non relevance to the business problem.
movie_combo = pd.merge(df_movies, df_ratings, left_on='movieId', right_on='movieId', how='left')

movie_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100854 entries, 0 to 100853
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100854 non-null  int64  
 1   title      100854 non-null  object 
 2   genres     100854 non-null  object 
 3   userId     100836 non-null  float64
 4   rating     100836 non-null  float64
 5   timestamp  100836 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 5.4+ MB


In [11]:
movie_combo.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [12]:
movie_combo.drop(['genres'], axis=1)

,movieId,title,userId,rating,timestamp
0,1,Toy Story (1995),1.0,4.0,9.649827e+08
1,1,Toy Story (1995),5.0,4.0,8.474350e+08
2,1,Toy Story (1995),7.0,4.5,1.106636e+09
3,1,Toy Story (1995),15.0,2.5,1.510578e+09
4,1,Toy Story (1995),17.0,4.5,1.305696e+09
...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),184.0,3.5,1.537110e+09
100851,193585,Flint (2017),184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),184.0,3.5,1.537110e+09


In [13]:
# Filling in missing timestamp values with the mode. 
movie_combo['timestamp'] = movie_combo['timestamp'].fillna(movie_combo['timestamp'].mode()[0])
movie_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100854 entries, 0 to 100853
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100854 non-null  int64  
 1   title      100854 non-null  object 
 2   genres     100854 non-null  object 
 3   userId     100836 non-null  float64
 4   rating     100836 non-null  float64
 5   timestamp  100854 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 5.4+ MB


In [14]:
movie_combo['timestamp'] = movie_combo['timestamp'].astype(int)

In [15]:
movie_combo.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1305696483


In [16]:
movie_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100854 entries, 0 to 100853
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100854 non-null  int64  
 1   title      100854 non-null  object 
 2   genres     100854 non-null  object 
 3   userId     100836 non-null  float64
 4   rating     100836 non-null  float64
 5   timestamp  100854 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 5.4+ MB


In [17]:
movie_combo.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1305696483


In [18]:
# converting timestamp to datetime index format


movie_combo['timestamp'] = movie_combo['timestamp'].apply(lambda t: datetime.utcfromtimestamp(t*1).strftime('%Y-%m-%d'))
movie_combo.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,2000-07-30
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,1996-11-08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,2005-01-25
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,2017-11-13
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,2011-05-18


# Train Test Split

In [ ]:
#reader = Reader(rating_scale =(0.5, 5) ) 
#df = Dataset.load_from_df(new_df,reader)

In [19]:
data = movie_combo[['userId', 'movieId', 'rating']]
reader = Reader(line_format='user item rating', sep=',',rating_scale=(0.5, 5))
data = Dataset.load_from_df(data,reader =reader)
# train-test-split
#trainset, testset = train_test_split(data, test_size=.2)
trainset, testset = surprise.model_selection.train_test_split(data, test_size=0.2, random_state=42)

# Baseline/ Dummy Model  KNN model

In [28]:
knn_model = KNNBasic().fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [29]:
knn_model

In [30]:
knn_model.sim

array([[1.        , 0.54183267, 0.40145103, ..., 0.        , 0.        ,
        0.        ],
       [0.54183267, 1.        , 0.38062756, ..., 0.        , 0.        ,
        0.        ],
       [0.40145103, 0.38062756, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [42]:
knn_model.test(testset)


[Prediction(uid=184.0, iid=105653, r_ui=4.0, est=4.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=105.0, iid=5617, r_ui=4.0, est=3.9126961921268997, details={'actual_k': 18, 'was_impossible': False}),
 Prediction(uid=182.0, iid=5423, r_ui=4.0, est=2.491690451715915, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid=288.0, iid=7293, r_ui=3.0, est=3.57912473658739, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid=75.0, iid=2628, r_ui=1.0, est=3.0049754540165092, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=606.0, iid=2541, r_ui=4.0, est=3.3568710795207433, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=79.0, iid=858, r_ui=5.0, est=4.643473312584755, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=288.0, iid=2391, r_ui=4.0, est=3.586720929085241, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid=182.0, iid=6215, r_ui=3.0, est=3.0, details={'actual_k'

In [43]:
surprise.accuracy.mae(knn_model.test(testset))

MAE:  nan


nan

In [44]:
print(accuracy.rmse( model.test(testset)))

RMSE: nan
nan


# Encoding 

One Hot Encoding since Genre is a Categorical Column. 

In [ ]:
#categorical_vars = ["genres"]
#one_hot_encoder = OneHotEncoder(sparse=False, drop = "first")

In [ ]:
#encoder_vars_array = one_hot_encoder.fit_transform(new_df[categorical_vars])
#encoder_feature_names = one_hot_encoder.get_feature_names(categorical_vars)
#encoder_vars_df = pd.DataFrame(encoder_vars_array, columns = encoder_feature_names)
#Data = pd.concat([new_df.reset_index(drop=True), encoder_vars_df.reset_index(drop=True)], axis = 1)
#Data.drop(categorical_vars, axis = 1, inplace = True)

# EDA and Visualizations 

In [ ]:
print('Number of users: ',len(Data.userId.unique()))
print('Number of movie titles: ',len(Data.movieId.unique()))
print('Number of ratings: ' ,Data.shape[0])
print('Number of tags: ' ,Data.tag.count())


In [ ]:
# groupby tags 
group = Data.groupby('tag').count()




group[:10].rating.plot(kind='bar', figsize=(15,6))



plt.title('Top 10 movie tags ');

In [ ]:
#top 10 re-occuring tags
top_tags =Data.groupby(by="tag").count().sort_values('movieId', ascending=False).userId.head(10)

top_tags


In [ ]:
# Distribution of ratings
Data.rating.plot.hist(bins=12)
plt.xlabel('Rating')
plt.title('Rating');